In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import os
import pandas as pd
from time import time

In [ ]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

In [ ]:
from GetAgentData import *

In [ ]:
#overview of the data
dataDirectory = os.path.join(os.getcwd(),'data')
DATA_FILE_PATH = os.path.join(dataDirectory,'100_50')

dir_list = os.listdir(DATA_FILE_PATH)
max_data_points = len(dir_list)
print("Number of data points: {}".format(len(dir_list)))
print(dir_list[0])




In [ ]:
class TopOptSequence:
    def __init__(self,ID,forces,dof,passive,x,numIterations):
        self.ID = ID
        self.forceImage = forces
        self.anchorImage = dof
        self.filledAreaImage = passive
        self.xPhys_array = x
        self.numIterations = numIterations
        self.iterationJumpTracker = []
    
    def dispenceData(self,iterationJump:int=5):
        """
        When called creates list of numpy arrays filled with the data needed to train the model

        returns:
            forces_array
            support_array
            filled_array
            x_array
            x_optimized_array
            finished_array
        """
        dataX = []
        dataY = []
        self.iterationJumpTracker = []
        for j in range(self.numIterations-iterationJump):
                dataX.append([self.forceImage.copy(),self.anchorImage.copy(),self.filledAreaImage.copy(),self.xPhys_array[:,:,j]])
                v = 0.0
                f= 'unfinished'
                if(j+iterationJump >= self.numIterations - 1):
                    v = 1.0
                    f = 'finished'
                dataY.append([self.xPhys_array[:,:,j+iterationJump],np.array([v])])
                self.iterationJumpTracker.append([j,j+iterationJump])

                #print("Adding itter: {} -> {}:{}".format(j,j+iterationJump,f))

        for j in range(1,min(iterationJump,self.numIterations)):
            # add the last iterations(dataY has True)
            dataX.append([self.forceImage.copy(),self.anchorImage.copy(),self.filledAreaImage.copy(),self.xPhys_array[:,:,-j -1]])
            dataY.append([self.xPhys_array[:,:,self.numIterations-1],np.array([1.])])
            self.iterationJumpTracker.append([-j,self.numIterations-1])

            #print("Adding itter: {} -> {}:finished".format(numIterations-j-1,numIterations-1))

        # add the optimal Stoping point data, input = output
        dataX.append([self.forceImage.copy(),self.anchorImage.copy(),self.filledAreaImage.copy(),self.xPhys_array[:,:,self.numIterations-1]])
        dataY.append([self.xPhys_array[:,:,self.numIterations-1],np.array([1.])])
        self.iterationJumpTracker.append([self.numIterations-1,self.numIterations-1])


        forces_array = []
        support_array = []
        filled_array = []
        x_array = []
        for forces,support,filled,x in dataX:

            forces_array.append(forces)
            support_array.append(support)
            filled_array.append(filled)
            x_array.append(x)

        x_optimized_array = []
        finished_array = []
        for x,finished in dataY:
            x_optimized_array.append(x)
            finished_array.append(finished)


        return forces_array,support_array,filled_array,x_array,x_optimized_array,finished_array

    def findNextIterations(self,i):
        """
        Given and index for a predicted image, find it's next optimized output.
        each output image will have a order that matches the order of it's input.
        By knowing the input image we can find what image the prediction was supposed to be.
        By knowing what the image was supposed to be, we can find next step in the iteration.


        Process works like this:
            - iterationTracker stores the input and output image,
            - We know the input image so we use this to find the correct output image
            - We then search the tracker for where the output image was used as input to get the correct next iteration.
        """
        def searchTracker(index):
            """given an input index find the matching output index"""
            for inputIndex,outputIndex in self.iterationJumpTracker:
                if(inputIndex == index):
                    return outputIndex
            return -1
        
        correctOutput = searchTracker(i)
        nextIteration = searchTracker(correctOutput)
        return nextIteration

    def formatPredictedData(self,predicted_x_array):
        """
        Given an array of images and the finished array, reformate the images with their respective inputs to create a new dataset that has as input the predicted x and as output the correct next iteration.

        The iteration Jump tracker has stored what iterations were jumped so by following the tracker list we can find the correct next output
        """

        n = len(predicted_x_array)
        if(n ==  self.numIterations):#check if correct number of iterations has been recieved
            forces_array = []
            support_array = []
            filled_array = []
            x_array = []
            x_optimized_array = []
            finished_array = []
            for i in range(n):
                correctOutput = self.findNextIterations(i)
                if(correctOutput <= 0):
                    continue
                else:
                    forces_array.append(self.forceImage.copy())
                    support_array.append(self.anchorImage.copy())
                    filled_array.append(self.filledAreaImage.copy())

                    x_array.append(predicted_x_array[i].copy())
                    v = 0.0
                    if(correctOutput >= n):
                        correctOutput = n-1
                        v = 1.0
                    x_optimized_array.append(self.xPhys_array[:,:,correctOutput])
                    finished_array.append([v])
                    
            return forces_array,support_array,filled_array,x_array,x_optimized_array,finished_array

        else:
            raise Exception("Data given is not of the correct format. required iterations: {}. iterations recieved: {}".format(self.numIterations,n))

    def dispenceFirstIterationData(self):
        dataX = []
        dataY = []
        
        dataX.append([self.forceImage.copy(),self.anchorImage.copy(),self.filledAreaImage.copy(),self.xPhys_array[:,:,0]])
        dataY.append([self.xPhys_array[:,:,1],np.array([0])])

        dataX.append([self.forceImage.copy(),self.anchorImage.copy(),self.filledAreaImage.copy(),self.xPhys_array[:,:,0]])
        dataY.append([self.xPhys_array[:,:,2],np.array([0])])

        forces_array = []
        support_array = []
        filled_array = []
        x_array = []
        for forces,support,filled,x in dataX:

            forces_array.append(forces)
            support_array.append(support)
            filled_array.append(filled)
            x_array.append(x)

        x_optimized_array = []
        finished_array = []
        for x,finished in dataY:
            x_optimized_array.append(x)
            finished_array.append(finished)


        return forces_array,support_array,filled_array,x_array,x_optimized_array,finished_array
        
    def dispenceM7modelData(self,iterationJump:int=5,predictionDepth:int=5):
        """
        When called creates list of numpy arrays filled with the data needed to train the model
        Designed to return the inital input but with an array of output representing the folowing iterations.

        This will allow the model to train on its own predictions
        """
        dataX = []
        dataY = []
        for j in range(self.numIterations-(iterationJump//2)):
            dataX.append([self.forceImage.copy(),self.anchorImage.copy(),self.filledAreaImage.copy(),self.xPhys_array[:,:,j]])
            
            dataY.append([])
            for i in range(predictionDepth):
                currentIteration = min(j + i*iterationJump,self.numIterations-1)
                v = 0.0
                if(currentIteration >= self.numIterations - 1):
                    v = 1.0
                dataY[j].append([self.xPhys_array[:,:,currentIteration],np.array([v])])


        forces_array = []
        support_array = []
        filled_array = []
        x_array = []
        for forces,support,filled,x in dataX:

            forces_array.append(forces)
            support_array.append(support)
            filled_array.append(filled)
            x_array.append(x)

        x_optimized_array = []
        finished_array = []
        for i in range(len(dataY)):
            x_optimized_array.append([])
            finished_array.append([])
            for x,finished in dataY[i]:
                x_optimized_array[i].append(x)
                finished_array[i].append(finished)


        return forces_array,support_array,filled_array,x_array,x_optimized_array,finished_array


In [ ]:
def buildDataSet(dataPointsToGrab:int):

    # Constants of interest
    # DATA_FILE_PATH = path to agent files
    # dir_List = all agent files
    # max_data_points = total number of datapoints

    dataPointsToGrab = min(dataPointsToGrab,max_data_points)

    #randomize the data grabed so that the first thee datapoints aren't always in the data.
    indexList = np.arange(max_data_points,dtype='int32')
    np.random.shuffle(indexList)

    sequenceData = []
    print("Retreiving {} Datapoints.".format(dataPointsToGrab))

    for i in range(dataPointsToGrab):
        print("{:.2f}%\t\t".format((100*(i/dataPointsToGrab))),end='\r')
        try:
            #join the data file path to a random sorted member within the data directory
            pathToAgent = os.path.join(DATA_FILE_PATH,dir_list[indexList[i]])
            forces,dof,passive,x,numIterations = formatIterativeModelDataSet(pathToAgent)
        except:
            #if an exception occurs list it and move forward
            print("Exception Occured at file '{}'.".format(os.path.join(DATA_FILE_PATH,dir_list[indexList[i]])))
            continue
        else:
            #if no error occured append that data to the data list
            sequenceData.append(TopOptSequence(i,forces,dof,passive,x,numIterations))

    print("100%\t\t")
    return sequenceData
        


In [ ]:
Data = buildDataSet(50)
print(len(Data))

In [ ]:
#check if the data output is correct
def checkArrays(arraysOfValues):
    correct = True
    for i in range(1,len(arraysOfValues)):
        a = np.equal(arraysOfValues[i-1],arraysOfValues[i]).sum()
        numberofValues = np.prod(arraysOfValues[i-1].shape)
        #print(a,numberofValues)
        if(a != numberofValues):
            print("iteration {} is not the same as iteration {}, {} != {}.".format(i-1,i,a,numberofValues))
            correct = False
        #print(a,forceValues)

    return correct
            
def plotIteration(input_array,output_array,finished_array):
    numIterations = len(input_array)
    for i in range(numIterations):
        fig,ax = plt.subplots(1,2)
        ax[0].imshow(input_array[i].T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
        ax[0].get_xaxis().set_visible(False)
        ax[0].get_yaxis().set_visible(False)
        ax[0].set_title("Input")

        ax[1].imshow(output_array[i].T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
        ax[1].get_xaxis().set_visible(False)
        ax[1].get_yaxis().set_visible(False)
        ax[1].set_title("Output, Finished:{}".format(finished_array[i]))
        
        plt.show()



def checkData(indexToUse):
    currentSum = 0
    for sequence in Data:
        currentSum += sequence.numIterations
    
    print("With {} problem statements there are {} sample datapoints.".format(len(Data),currentSum))

    forces_array,support_array,filled_array,x_array,x_optimized_array,finished_array = Data[indexToUse].dispenceFirstIterationData()
    #check forces
    print("check Forces:")
    if(checkArrays(forces_array)):
        print("\tOk.")

    print("check Supports:")
    if(checkArrays(support_array)):
        print("\tOk.")
    
    print("check Filled area:")
    if(checkArrays(filled_array)):
        print("\tOk.")
    
    numIterations = Data[indexToUse].numIterations
    print("Iterations:",numIterations )
    plotIteration(x_array,x_optimized_array,finished_array)


In [ ]:
#forces_array,support_array,filled_array,x_array,x_optimized_array,finished_array = Data[0].dispenceData()

#print(Data[0].numIterations)
#print(len(x_array))
checkData(0)

<h1>Model Information</h1>

Below are the models that will be used to attempt to learn the dataset

In [ ]:
#universal parameters
activation = 'relu'
uniformRandomInitalizer = tf.random_uniform_initializer(minval=-0.5, maxval=0.5)

In [ ]:
def buildModel_m7(x_inputShape = (100,50,1),forces_inputShape = (101,51,2),supports_inputShape = (101,51,1),filled_inputShape = (100,50,1)):
    partInput = keras.Input(shape=x_inputShape,name="x")
    forcesInput = keras.Input(shape=forces_inputShape,name="forces")
    supportsInput = keras.Input(shape=supports_inputShape,name="supports")
    #since filled input is solely the solid area it will be passed into the model at the very end
    filledInput = keras.Input(shape=filled_inputShape,name="filled")

    partInput_resize = layers.Resizing(height=128,width=64)(partInput)
    forcesInput_resize = layers.Resizing(height=128,width=64)(forcesInput)
    forcesInput_resize = layers.Activation(activation='tanh')(forcesInput_resize)# normaize the force input
    supportsInput_resize = layers.Resizing(height=128,width=64)(supportsInput)
    filledInput_resize = layers.Resizing(height=128,width=64)(filledInput)

    concatenatedStartLayer = layers.Concatenate()([partInput_resize,forcesInput_resize,supportsInput_resize,filledInput_resize])

    #First Convolution Layer
    conv_128_64 = layers.Conv2D(filters= 16, kernel_size=(3,3),padding='same',activation=activation)(concatenatedStartLayer)
    conv_128_64 = layers.Conv2D(filters= 16, kernel_size=(3,3),padding='same',activation=activation)(conv_128_64)
    conv_64_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_128_64)
    conv_64_32 = layers.GaussianNoise(stddev=0.1)(conv_64_32)
    conv_64_32 = layers.Dropout(rate=0.1)(conv_64_32)

    #Second convolution Layer
    conv_64_32 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(conv_64_32)
    conv_64_32 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(conv_64_32)
    conv_32_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_64_32)
    conv_32_16 = layers.GaussianNoise(stddev=0.1)(conv_32_16)
    conv_32_16 = layers.Dropout(rate=0.2)(conv_32_16)

    conv_32_16 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(conv_32_16)
    conv_32_16 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(conv_32_16)
    conv_16_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_32_16)
    conv_16_8 = layers.Dropout(rate=0.3)(conv_16_8)

    conv_16_8 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(conv_16_8)
    conv_16_8 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(conv_16_8)

    #Dense 2D layer
    newShape=conv_16_8.shape[1:]
    shapeFlat = np.prod(newShape)
    denseLayer = layers.Flatten()(conv_16_8)
    denseLayer = layers.Dense(shapeFlat,activation=activation)(denseLayer)
    denseLayer_16_8 = layers.Reshape(newShape)(denseLayer)

    #upscaleLayer
    #upscaling is performed by convolution transpose where stride=2 < kernalsize
    convUpscale_32_16 = layers.Conv2DTranspose(filters= 32, kernel_size=(5,5),strides=2,padding='same',activation=activation)(denseLayer_16_8)
    convUpscale_32_16 = layers.Dropout(rate=0.3)(convUpscale_32_16)
    convUpscale_32_16 = layers.GaussianNoise(stddev=0.1)(convUpscale_32_16)
    convUpscale_32_16 = layers.Concatenate()([convUpscale_32_16,conv_32_16])
    convUpscale_32_16 = layers.Conv2D(filters = 64, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_32_16)

    convUpscale_64_32 = layers.Conv2DTranspose(filters= 64, kernel_size=(5,5),strides=2,padding='same',activation=activation)(convUpscale_32_16)
    convUpscale_64_32 = layers.Dropout(rate=0.2)(convUpscale_64_32)
    convUpscale_64_32 = layers.GaussianNoise(stddev=0.1)(convUpscale_64_32)
    convUpscale_64_32 = layers.Concatenate()([convUpscale_64_32,conv_64_32])
    convUpscale_64_32 = layers.Conv2D(filters = 64, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_64_32)

    convUpscale_128_64 = layers.Conv2DTranspose(filters= 64, kernel_size=(5,5),strides=2,padding='same',activation=activation)(convUpscale_64_32)
    convUpscale_128_64 = layers.Dropout(rate=0.1)(convUpscale_128_64)
    convUpscale_128_64 = layers.Concatenate()([convUpscale_128_64,conv_128_64])
    convUpscale_128_64 = layers.Conv2D(filters = 64, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_128_64)

    output_resize =layers.Resizing(height=100,width=50)(convUpscale_128_64)
    output_part = layers.Conv2D(filters= 1, kernel_size=(1,1),padding='same',activation='hard_sigmoid', name="x_out")(output_resize)
    """
    The hard sigmoid activation, defined as:
        if x < -2.5: return 0
        if x > 2.5: return 1
        if -2.5 <= x <= 2.5: return 0.2 * x + 0.5
    """

    return keras.Model(inputs= [partInput,forcesInput,supportsInput,filledInput],outputs=[output_part])#,finishedOutput])

class Model_m7(keras.Model):
    def __init__(self):

        super(Model_m7, self).__init__()
        self.model = buildModel_m7()
        

    def call(self,data,training = False):
        #part = data['x']
        #forces = data['forces']
        #supports = data['supports']
        #filled = data['filled']
        #print(1)
        if(training):
            
            out1 = self.model(data)
            data['x'] = out1
            
            #print(2)
            out2 = self.model(data)
            data['x'] = out2

            out3 = self.model(data)
            data['x'] = out3
        
            out4 = self.model(data)
            data['x'] = out4

            out5 = self.model(data)
            #print(6)
            return out1,out2,out3,out4,out5
        else:
            return self.model(data)







In [ ]:
def buildModel_m8(x_inputShape = (100,50,1),forces_inputShape = (101,51,2),supports_inputShape = (101,51,1),filled_inputShape = (100,50,1)):
    partInput = keras.Input(shape=x_inputShape,name="x")
    forcesInput = keras.Input(shape=forces_inputShape,name="forces")
    supportsInput = keras.Input(shape=supports_inputShape,name="supports")
    #since filled input is solely the solid area it will be passed into the model at the very end
    filledInput = keras.Input(shape=filled_inputShape,name="filled")

    partInput_resize = layers.Resizing(height=128,width=64)(partInput)
    forcesInput_resize = layers.Resizing(height=128,width=64)(forcesInput)
    forcesInput_resize = layers.Activation(activation='tanh')(forcesInput_resize)# normaize the force input
    supportsInput_resize = layers.Resizing(height=128,width=64)(supportsInput)
    filledInput_resize = layers.Resizing(height=128,width=64)(filledInput)

    concatenatedStartLayer = layers.Concatenate()([partInput_resize,forcesInput_resize,supportsInput_resize,filledInput_resize])

    #First Convolution Layer
    conv_128_64 = layers.Conv2D(filters= 16, kernel_size=(3,3),padding='same',activation=activation)(concatenatedStartLayer)
    conv_128_64 = layers.Conv2D(filters= 16, kernel_size=(3,3),padding='same',activation=activation)(conv_128_64)
    conv_64_32 = layers.MaxPooling2D(pool_size=(2,2))(conv_128_64)
    conv_64_32 = layers.GaussianNoise(stddev=0.1)(conv_64_32)

    #Second convolution Layer
    conv_64_32 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(conv_64_32)
    conv_64_32 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(conv_64_32)
    conv_32_16 = layers.MaxPooling2D(pool_size=(2,2))(conv_64_32)
    conv_32_16 = layers.GaussianNoise(stddev=0.1)(conv_32_16)

    conv_32_16 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(conv_32_16)
    conv_32_16 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(conv_32_16)
    conv_16_8 = layers.MaxPooling2D(pool_size=(2,2))(conv_32_16)
    conv_16_8 = layers.GaussianNoise(stddev=0.1)(conv_16_8)

    conv_16_8 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(conv_16_8)
    conv_16_8 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(conv_16_8)

    #upscaleLayer
    #upscaling is performed by convolution transpose where stride=2 < kernalsize
    convUpscale_32_16 = layers.Conv2DTranspose(filters= 64, kernel_size=(5,5),strides=2,padding='same',activation=activation)(conv_16_8)
    convUpscale_32_16 = layers.GaussianNoise(stddev=0.1)(convUpscale_32_16)
    convUpscale_32_16 = layers.Concatenate()([convUpscale_32_16,conv_32_16])
    convUpscale_32_16 = layers.Conv2D(filters = 64, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_32_16)
    convUpscale_32_16 = layers.Conv2D(filters = 64, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_32_16)

    convUpscale_64_32 = layers.Conv2DTranspose(filters= 32, kernel_size=(5,5),strides=2,padding='same',activation=activation)(convUpscale_32_16)
    convUpscale_64_32 = layers.GaussianNoise(stddev=0.1)(convUpscale_64_32)
    convUpscale_64_32 = layers.Concatenate()([convUpscale_64_32,conv_64_32])
    convUpscale_64_32 = layers.Conv2D(filters = 32, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_64_32)
    convUpscale_64_32 = layers.Conv2D(filters = 32, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_64_32)

    convUpscale_128_64 = layers.Conv2DTranspose(filters= 64, kernel_size=(5,5),strides=2,padding='same',activation=activation)(convUpscale_64_32)
    convUpscale_64_32 = layers.GaussianNoise(stddev=0.1)(convUpscale_64_32)
    convUpscale_128_64 = layers.Concatenate()([convUpscale_128_64,conv_128_64])
    convUpscale_128_64 = layers.Conv2D(filters = 16, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_128_64)
    convUpscale_128_64 = layers.Conv2D(filters = 16, kernel_size=(3,3),strides=1,padding='same',activation=activation)(convUpscale_128_64)

    output_resize =layers.Resizing(height=100,width=50)(convUpscale_128_64)
    output_part = layers.Conv2D(filters= 1, kernel_size=(1,1),padding='same',activation='hard_sigmoid', name="x_out")(output_resize)
    """
    The hard sigmoid activation, defined as:
        if x < -2.5: return 0
        if x > 2.5: return 1
        if -2.5 <= x <= 2.5: return 0.2 * x + 0.5
    """

    return keras.Model(inputs= [partInput,forcesInput,supportsInput,filledInput],outputs=[output_part])#,finishedOutput])

class Model_m8(keras.Model):
    def __init__(self):

        super(Model_m8, self).__init__()
        self.model = buildModel_m8()
        

    def call(self,data,training = False):
        #part = data['x']
        #forces = data['forces']
        #supports = data['supports']
        #filled = data['filled']
        #print(1)
        if(training):
            
            out1 = self.model(data)
            data['x'] = out1
            
            #print(2)
            out2 = self.model(data)
            data['x'] = out2

            out3 = self.model(data)
            data['x'] = out3
        
            out4 = self.model(data)
            data['x'] = out4

            out5 = self.model(data)
            #print(6)
            return out1,out2,out3,out4,out5
        else:
            return self.model(data)







In [ ]:
def SetUpOptimizer(variant):
    """
    Builds a keras optmizer based of default parameters
    
    Accepts:
        1:adam
        2:adadelta
        3:adafactor
        4:adagrad
        5:adamax
        6:ftrl
        7:nadam
        8:rmsprop
    """
    if(variant == 1 or variant == 'adam'):
        print("Optimizer: Adam")
        return keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam') 
    elif(variant == 2 or variant == 'adadelta'):
        print("Optimizer: AdaDelta")
        return keras.optimizers.experimental.Adadelta(
                                                        learning_rate=0.001,
                                                        rho=0.95,
                                                        epsilon=1e-07,
                                                        ema_momentum=0.99,
                                                        name='Adadelta'
                                                    )
    elif(variant == 3 or variant == 'adafactor'):
        print("Optimizer: AdaFactor")
        return keras.optimizers.experimental.Adafactor(
                                                        learning_rate=0.001,
                                                        beta_2_decay=-0.8,
                                                        epsilon_1=1e-30,
                                                        epsilon_2=0.001,
                                                        clip_threshold=1.0,
                                                        ema_momentum=0.99,
                                                        name='Adafactor'
                                                    )
    elif(variant == 4 or variant == 'adagrad'):
        print("Optimizer: AdaGrad")
        return keras.optimizers.experimental.Adagrad(
                                                        learning_rate=0.001,
                                                        initial_accumulator_value=0.1,
                                                        epsilon=1e-07,
                                                        ema_momentum=0.99,
                                                        name='Adagrad'
                                                    )
    elif(variant == 5 or variant == 'adamax'):
        print("Optimizer: AdaMax")
        return keras.optimizers.experimental.Adamax(
                                                        learning_rate=0.001,
                                                        beta_1=0.9,
                                                        beta_2=0.999,
                                                        epsilon=1e-07,
                                                        ema_momentum=0.99,
                                                        name='Adamax'
                                                    )
    elif(variant == 6 or variant == 'ftrl'):
        print("Optimizer: FTRL")
        return keras.optimizers.experimental.Ftrl(
                                                    learning_rate=0.001,
                                                    learning_rate_power=-0.5,
                                                    initial_accumulator_value=0.1,
                                                    l1_regularization_strength=0.0,
                                                    l2_regularization_strength=0.0,
                                                    l2_shrinkage_regularization_strength=0.0,
                                                    beta=0.0,
                                                    ema_momentum=0.99,
                                                    name='Ftrl'
                                                )
    elif(variant == 7 or variant == 'nadam'):
        print("Optimizer: Nadam")
        return keras.optimizers.experimental.Nadam(
                                                    learning_rate=0.001,
                                                    beta_1=0.9,
                                                    beta_2=0.999,
                                                    epsilon=1e-07,
                                                    ema_momentum=0.99,
                                                    name='Nadam'
                                                )
    elif(variant == 8 or variant == 'rmsprop'):
        print("Optimizer: RMSprop")
        return keras.optimizers.experimental.RMSprop(
                                                        learning_rate=0.001,
                                                        rho=0.9,
                                                        momentum=0.0,
                                                        epsilon=1e-07,
                                                        ema_momentum=0.99,
                                                        ema_overwrite_frequency=100,
                                                        name='RMSprop'
                                                    )
    else:
        print("Optimizer: Adam")
        return keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam') 

In [ ]:
#setUp modelSaving

def getModel(modelNumber,optimizerVarient:int = 1):
    if(modelNumber == 7):
        model = Model_m7()
        fileSaveName = "Model_m7"
    elif(modelNumber == 8):
        model = Model_m8()
        fileSaveName = "Model_m8"
    else:
        raise Exception("No model identified, model {} DNE.".format(modelNumber))
    

    modelPath = os.path.join(os.getcwd(),'ModelSave',fileSaveName)
    
    cp_callback = keras.callbacks.ModelCheckpoint(filepath=os.path.join(modelPath,fileSaveName),
                                                     save_weights_only=True,
                                                     verbose=1)
    if(os.path.isdir(modelPath)):
        try:
            
            model.load_weights(os.path.join(modelPath,fileSaveName))
        except:
            print("Model weights could not be loaded.")
        else:
            print("Model weights Loaded")
    else:
        os.mkdir(modelPath)
        print("Model path created")

    
    model.compile(  optimizer=SetUpOptimizer(optimizerVarient),
                        loss= keras.losses.BinaryCrossentropy()
                        )
    return model,cp_callback

In [ ]:
currentModelNumber = 8 #change this one
model,callBack = getModel(currentModelNumber)
print()
model.model.summary()

In [ ]:
def EditImage(image):

    #noise = np.random.normal(loc=0,scale=.05,size=image.shape)
    #return image + noise

    return image#np.flip(image,axis=0)

In [ ]:
def DrawModelIteratins(num):
    sequenceToStart:TopOptSequence = Data[num]
    numImages = sequenceToStart.numIterations
    imagesToShow = 5
    #print(numImages)
    fig,ax = plt.subplots(2,imagesToShow)

    sequenceToStart.dispenceData()

    nelx = 100
    nely = 50
    force,support,filled,x_start,x_optimized,finished = sequenceToStart.dispenceData(1)

    force = np.array([force[0]])
    force = EditImage(force)
    support = EditImage(np.array([support[0]]))
    filled = EditImage(np.array([filled[0]]))
    ImageToPredict = np.array([sequenceToStart.xPhys_array[:,:,0]])
    PredictedImages = [ImageToPredict]

    start = time()
    for i in range(numImages):
        
        output = model.predict({'x':ImageToPredict,'forces':force,'supports':support,'filled':filled},verbose = 0)
        ImageToPredict = output#[0]
        PredictedImages.append(ImageToPredict)
    end = time()
    print("{} iterations took {:.2f} seconds or about {:.5f} seconds per iteration.".format(numImages,end-start,(end-start)/numImages))
    imagesToJump = numImages // imagesToShow

    for i in range(0,imagesToShow-1):
        ax[0,i].imshow(np.reshape(sequenceToStart.xPhys_array[:,:,i*imagesToJump],(nelx,nely)).T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
        ax[0,i].get_xaxis().set_visible(False)
        ax[0,i].get_yaxis().set_visible(False)

        #ax[1,i].set_title("Pred")#:{}".format(finalBit(Y_score_finished[rnd[i]])))
        ax[1,i].imshow(np.reshape(PredictedImages[i*imagesToJump],(nelx,nely)).T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
        ax[1,i].get_xaxis().set_visible(False)
        ax[1,i].get_yaxis().set_visible(False)
    ax[0,-1].imshow(np.reshape(sequenceToStart.xPhys_array[:,:,-1],(nelx,nely)).T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
    ax[0,-1].get_xaxis().set_visible(False)
    ax[0,-1].get_yaxis().set_visible(False)

    #ax[1,i].set_title("Pred")#:{}".format(finalBit(Y_score_finished[rnd[i]])))
    ax[1,-1].imshow(np.reshape(PredictedImages[-1],(nelx,nely)).T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
    ax[1,-1].get_xaxis().set_visible(False)
    ax[1,-1].get_yaxis().set_visible(False)

    ax[0,0].set_title("True")
    ax[1,0].set_title("Pred")
    plt.show()


In [ ]:
n = 0#np.random.randint(0,len(Data_score)-1)
print(n)
DrawModelIteratins(n)

In [ ]:
from PIL import Image
import io

In [ ]:
def plotOverIterations(index,data,overIterateFactor:int=4):
    sequenceToStart = data[index]
    numImages = sequenceToStart.numIterations
    imagesToShow = numImages
    #print(numImages)

    sequenceToStart.dispenceData()

    nelx = 100
    nely = 50
    force,support,filled,x_start,x_optimized,finished = sequenceToStart.dispenceData(1)

    force = np.array([force[0]])
    support = np.array([support[0]])
    filled = np.array([filled[0]])
    ImageToPredict = np.array([sequenceToStart.xPhys_array[:,:,0]])
    PredictedImages = [ImageToPredict]


    start = time()
    for i in range(numImages * overIterateFactor):
        
        output = model.predict({'x':ImageToPredict,'forces':force,'supports':support,'filled':filled},verbose = 0)
        ImageToPredict = output#[0]
        PredictedImages.append(ImageToPredict)
    end = time()
    print("{} iterations took {:.2f} seconds or about {:.5f} seconds per iteration.".format(numImages * overIterateFactor,end-start,(end-start)/(numImages * overIterateFactor)))
    imagesToJump = 1
    imageArray = []

    for i,image in enumerate(PredictedImages):
        fig,ax = plt.subplots(1,1)
        
        if(i == 0):
            ax.set_title("Iteration: {}".format(i))
        else:
            im1 = np.reshape(PredictedImages[i],(nelx*nely))
            im2 = np.reshape(PredictedImages[i-1],(nelx*nely))
            ax.set_title("Iteration: {}, Change: {:.5f}".format(i,np.linalg.norm(im1-im2,ord=np.inf)))
        ax.imshow(np.reshape(image,(nelx,nely)).T,cmap='gray_r',norm=colors.Normalize(vmin=0,vmax=1))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        img_buf = io.BytesIO()
        plt.savefig(img_buf, format='png')

        im = Image.open(img_buf)
        imageArray.append(im)

        #plt.show()
    return imageArray

In [ ]:

i=0
im_array = plotOverIterations(i,Data,10)
im = im_array.pop(0)
im.save("out{}.gif".format(i),save_all=True,append_images = im_array,optimize=False,loop=0)
im.close()

In [ ]:
print(model.model.input)

In [ ]:
for layer in model.model.layers:
#layer = model.model.layers[1].output
    name = layer.output.name
    #if('conv2d' in name):
    print(name)
print(len(model.model.layers))

In [ ]:

seq = Data[0]
force,support,filled,x_start,x_optimized,finished = seq.dispenceData(1)

force = np.array([force[0]])
support = np.array([support[0]])
filled = np.array([filled[0]])
ImageToPredict = np.array([seq.xPhys_array[:,:,0]])
PredictedImages = [ImageToPredict]

layer_output=model.model.layers[9].output
intermediate_model=tf.keras.models.Model(inputs=model.model.input,outputs=layer_output)
print(intermediate_model)

intermediate_prediction=intermediate_model.predict({'x':ImageToPredict,'forces':force,'supports':support,'filled':filled})




In [ ]:
#Visualize the model

def showModelFilters(layerNum,Seqence:TopOptSequence,iterationIdex:int=0):
    """
    Show the most activated layers of the model filters as the selected image is passed through the model
    """
    #cap off the index
    iterationIdex = min(Seqence.numIterations-1,max(0,int(iterationIdex)))

    seq = Data[0]
    force,support,filled,x_start,x_optimized,finished = seq.dispenceData(1)

    force = np.array([force[0]])
    support = np.array([support[0]])
    filled = np.array([filled[0]])
    ImageToPredict = np.array([seq.xPhys_array[:,:,iterationIdex]])
    PredictedImages = [ImageToPredict]

    layer_output=model.model.layers[layerNum].output
    intermediate_model=tf.keras.models.Model(inputs=model.model.input,outputs=layer_output)
    print(intermediate_model)

    intermediate_prediction=intermediate_model.predict({'x':ImageToPredict,'forces':force,'supports':support,'filled':filled})
    return intermediate_prediction
    



In [ ]:

def plotModelFilters(lNum,index):

    intermediate_prediction = showModelFilters(lNum,Data[0],index)
    #print(intermediate_prediction.shape)
    im = np.reshape(intermediate_prediction,newshape=intermediate_prediction.shape[1:])
    #print(im.shape)

    nfilters = im.shape[-1]
    print("Layer {}({}) has {} filters.".format(model.model.layers[lNum].output.name,lNum,nfilters))
    print("Shape={}".format(im.shape))
    #print(nfilters)
    if(nfilters == 1):
        fix,ax = plt.subplots(1,1)
        
        ax.imshow(im[:,:,0].T,norm=colors.Normalize(vmin=0))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    elif(nfilters//4 == 0):
        fix,ax = plt.subplots(1,nfilters)
        for j in range(nfilters):
            ax[j].imshow(im[:,:,j].T,norm=colors.Normalize(vmin=0))
            ax[j].get_xaxis().set_visible(False)
            ax[j].get_yaxis().set_visible(False)
    else:
        for i in range(nfilters//4):
            fix,ax = plt.subplots(1,4)
            for j in range(4):
                ax[j].imshow(im[:,:,4*i + j].T,norm=colors.Normalize(vmin=0))
                ax[j].get_xaxis().set_visible(False)
                ax[j].get_yaxis().set_visible(False)

            plt.show()

In [ ]:
numLayers = len(model.model.layers)
firstLayer = 9

for lNum in range(firstLayer,numLayers):
    print('='*50)
    plotModelFilters(lNum,0)